In [4]:
import matplotlib.pyplot as plt

In [3]:
import gym
import numpy as np
from agent import Agent
import tensorflow as tf
from datetime import datetime
from agent import Agent
import pandas as pd
import numpy as np
import gymnasium as gym
import datetime
import gym_trading_env
from gym_trading_env.downloader import download
from gym_trading_env.renderer import Renderer
import matplotlib.pyplot as plt
from src.stock_trading_env.single_stock_env import SingleStockTradingEnv
from stable_baselines3 import PPO
from stable_baselines3.common.policies import ActorCriticPolicy
from typing import Callable, Dict, List, Optional, Tuple, Type, Union

import gym
import torch as th
from torch import nn


In [8]:
import pandas_ta as ta
from sklearn.preprocessing import MinMaxScaler
df = pd.read_csv("examples/data/AAPL.csv", parse_dates=["date"], index_col= "date")
df.sort_index(inplace= True)
df.dropna(inplace= True)
df.drop_duplicates(inplace=True)

# Generating features
# WARNING : the column names need to contain keyword 'feature' !
df["feature_close"] = df["close"].pct_change()
df["feature_open"] = df["open"]/df["close"]
df["feature_high"] = df["high"]/df["close"]
df["feature_low"] = df["low"]/df["close"]
# df["feature_volume"] = df["Volume USD"] / df["Volume USD"].rolling(7*24).max()

rsi_values = ta.rsi(df["close"]).values.reshape(-1, 1)
scaler = MinMaxScaler()
df["feature_rsi"] = scaler.fit_transform(rsi_values)
df = df['2020-01-01':]
df.dropna(inplace= True)


In [157]:
df

,open,high,low,close,volume,feature_close,feature_open,feature_high,feature_low,feature_rsi
date,,,,,,,,,,
2020-01-02 21:30:00+07:00,74.0600,75.150000,73.7975,75.0875,135647456.0,0.022816,0.986316,1.000832,0.982820,0.909138
2020-01-03 21:30:00+07:00,74.2875,75.145000,74.1250,74.3575,146535512.0,-0.009722,0.999059,1.010591,0.996873,0.818799
2020-01-06 21:30:00+07:00,73.4475,74.990000,73.1875,74.9500,118578576.0,0.007968,0.979953,1.000534,0.976484,0.838232
2020-01-07 21:30:00+07:00,74.9600,75.225000,74.3700,74.5975,111510620.0,-0.004703,1.004859,1.008412,0.996950,0.795818
2020-01-08 21:30:00+07:00,74.2900,76.109975,74.2890,75.7975,132363784.0,0.016086,0.980111,1.004122,0.980098,0.836896
...,...,...,...,...,...,...,...,...,...,...
2023-07-25 20:30:00+07:00,193.3300,194.440000,192.9150,193.6200,37283201.0,0.004514,0.998502,1.004235,0.996359,0.622804
2023-07-26 20:30:00+07:00,193.6700,195.640000,193.3200,194.5000,47471868.0,0.004545,0.995733,1.005861,0.993933,0.646923
2023-07-27 20:30:00+07:00,196.0200,197.200000,192.5500,193.2200,47460180.0,-0.006581,1.014491,1.020598,0.996532,0.586048


In [9]:
import gymnasium as gym
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv

env = gym.make(
        "single-stock-v0",
        name="BTCUSD",
        df=df,
        windows=1,
        positions=[0,0.5,1],  # From -1 (=SHORT), to +1 (=LONG)
        initial_position=0,  #Initial position
        trading_fees=0,  # 0.01% per stock buy / sell
        # borrow_interest_rate=0.0003/100,  #per timestep (= 1h here)
        # reward_function=reward_function,
        portfolio_initial_value=100000,  # in FIAT (here, USD)
        max_episode_duration='max',
        verbose=1
    )
env.add_metric("Reward", lambda history: history["reward"][-1])
env.add_metric(
    "Portfolio Valuation", lambda history: history["portfolio_valuation"][-1]
)

/home/praew/miniconda3/envs/ppo/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.add_metric to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.add_metric` for environment variables or `env.get_wrapper_attr('add_metric')` that will search the reminding wrappers.
  logger.warn(


In [4]:
from typing import Callable, Dict, List, Optional, Tuple, Type, Union

import gym
import torch as th
from torch import nn
from stable_baselines3.common.policies import ActorCriticPolicy


class Transpose(nn.Module):
    def forward(self, x):
        batch_size = x.shape[0]
        return x.view(batch_size, -1)

class CustomNetwork(nn.Module):
    """
    Custom network for policy and value function.
    It receives as input the features extracted by the feature extractor.

    :param feature_dim: dimension of the features extracted with the features_extractor (e.g. features from a CNN)
    :param last_layer_dim_pi: (int) number of units for the last layer of the policy network
    :param last_layer_dim_vf: (int) number of units for the last layer of the value network
    """

    def __init__(
        self,
        feature_dim: int,
        timesteps: int = 12,
        last_layer_dim_pi: int = 64,
        last_layer_dim_vf: int = 64,
    ):
        super(CustomNetwork, self).__init__()

        # IMPORTANT:
        # Save output dimensions, used to create the distributions
        self.latent_dim_pi = last_layer_dim_pi
        self.latent_dim_vf = last_layer_dim_vf

        # Policy network
        self.policy_net = nn.Sequential(
            nn.Linear(last_layer_dim_pi*timesteps*feature_dim, last_layer_dim_pi*feature_dim), nn.ReLU(),
            nn.Linear(last_layer_dim_pi*feature_dim, last_layer_dim_pi), nn.Tanh()
        )
        # Value network
        self.value_net = nn.Sequential(
            nn.Linear(last_layer_dim_vf*timesteps*feature_dim, last_layer_dim_vf*feature_dim), nn.ReLU(),
            nn.Linear(last_layer_dim_vf*feature_dim, last_layer_dim_vf), nn.Tanh()
        )

    def forward(self, features: th.Tensor) -> Tuple[th.Tensor, th.Tensor]:
        """
        :return: (th.Tensor, th.Tensor) latent_policy, latent_value of the specified network.
            If all layers are shared, then ``latent_policy == latent_value``
        """
        return self.policy_net(features), self.value_net(features)

    def forward_actor(self, features: th.Tensor) -> th.Tensor:
        return self.policy_net(features)

    def forward_critic(self, features: th.Tensor) -> th.Tensor:
        return self.value_net(features)


class CustomActorCriticPolicy(ActorCriticPolicy):
    def __init__(
        self,
        observation_space: gym.spaces.Space,
        action_space: gym.spaces.Space,
        lr_schedule: Callable[[float], float],
        net_arch: Optional[List[Union[int, Dict[str, List[int]]]]] = None,
        activation_fn: Type[nn.Module] = nn.Tanh,
        *args,
        **kwargs,
    ):

        super(CustomActorCriticPolicy, self).__init__(
            observation_space,
            action_space,
            lr_schedule,
            net_arch,
            activation_fn,
            # Pass remaining arguments to base class
            *args,
            **kwargs,
        )
        # Disable orthogonal initialization
        self.ortho_init = False

    def _build_mlp_extractor(self) -> None:
        self.mlp_extractor = CustomNetwork(
        last_layer_dim_pi = 1,
        last_layer_dim_vf = 1,
        timesteps = self.observation_space.shape[1],
        feature_dim=1
        )


In [10]:
policy_kwargs = dict(activation_fn=th.nn.ReLU, net_arch=[256, 256])
model = PPO('MlpPolicy', env,policy_kwargs=policy_kwargs,verbose=0,n_epochs=5, tensorboard_log="./PPO_tensorboard/")
model_2 = PPO(CustomActorCriticPolicy, env, verbose=0)
# model_3 = PPO("CnnPolicy", env, verbose=0)

In [11]:
model_2.policy

CustomActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pi_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (vf_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): CustomNetwork(
    (policy_net): Sequential(
      (0): Linear(in_features=5, out_features=1, bias=True)
      (1): ReLU()
      (2): Linear(in_features=1, out_features=1, bias=True)
      (3): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=5, out_features=1, bias=True)
      (1): ReLU()
      (2): Linear(in_features=1, out_features=1, bias=True)
      (3): Tanh()
    )
  )
  (action_net): Linear(in_features=1, out_features=3, bias=True)
  (value_net): Linear(in_features=1, out_features=1, bias=True)
)

In [48]:
model.learn(total_timesteps=100_000)

Market Return : 161.63%   |   Portfolio Return : 348.66%   |   Reward : 0.00112844452345307   |   Portfolio Valuation : 448657.90000000014   |   
Market Return : 161.63%   |   Portfolio Return : 121.03%   |   Reward : 0.00309037262027712   |   Portfolio Valuation : 221026.5374999998   |   
Market Return : 161.63%   |   Portfolio Return : 150.38%   |   Reward : 0.0029759014620204427   |   Portfolio Valuation : 250380.46250000014   |   
Market Return : 161.63%   |   Portfolio Return : 37.12%   |   Reward : 0.002212136693653432   |   Portfolio Valuation : 137123.20000000004   |   
Market Return : 161.63%   |   Portfolio Return : 91.84%   |   Reward : 0.0   |   Portfolio Valuation : 191841.05000000013   |   
Market Return : 161.63%   |   Portfolio Return : 45.74%   |   Reward : 0.0023493995711029413   |   Portfolio Valuation : 145740.17500000005   |   
Market Return : 161.63%   |   Portfolio Return : 88.34%   |   Reward : 0.002967104824984842   |   Portfolio Valuation : 188341.24999999994 

In [49]:
model.policy

ActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pi_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (vf_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=5, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): ReLU()
    )
    (value_net): Sequential(
      (0): Linear(in_features=5, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): ReLU()
    )
  )
  (action_net): Linear(in_features=256, out_features=3, bias=True)
  (value_net): Linear(in_features=256, out_features=1, bias=True)
)

In [12]:
import numpy as np
from stable_baselines3.common.policies import obs_as_tensor

def predict_proba(model, state):
    obs = model.policy.obs_to_tensor(state)[0]
    dis = model.policy.get_distribution(obs)
    probs = dis.distribution.probs
    probs_np = probs.detach().numpy()
    return probs_np

In [55]:
score_history_pred = []
action_history = []
model = model
score_pred = 0
reward_tot = 0
######predict########
obs, info = env.reset()
done = False
truncated = False
while not done and not truncated:
    obs = obs[np.newaxis, ...]
    action, _ = model.predict(obs, deterministic=True)
    action_history.append(action[0])
    obs, reward, done, truncated, info = env.step(action[0])
    reward_tot += reward
    
print(reward_tot)



Market Return : 161.63%   |   Portfolio Return : 158.81%   |   Reward : 0.0031190979910628498   |   Portfolio Valuation : 258811.25   |   
0.9509288455547625


In [56]:
model.set_env(env)

In [57]:
evaluate(model, env, num_steps = 1000, isrender=True)

NameError: name 'evaluate' is not defined

In [47]:
print(action_history)

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [53]:
from stable_baselines3.common.env_checker import check_env
check_env(env, warn=True)

/home/praew/miniconda3/envs/ppo/lib/python3.11/site-packages/stable_baselines3/common/env_checker.py:238: UserWarning: Your observation  has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(


In [37]:
model.train()

In [39]:
model